In [387]:
import sys, os, io, json, numpy as np, random, time
#from sklearn.metrics import roc_auc_score
import xgboost as xgb
import tensorflow as tf
from importlib import reload
import split
reload(split)
from split import split_graph, split_interface, make_split, make_split_quick, make_gax, split_quick_graph
import graphviz
from sklearn import metrics
import cProfile


def profile(func):
    """Decorator for run function profile"""
    def wrapper(*args, **kwargs):
        profile_filename = func.__name__ + '.prof'
        profiler = cProfile.Profile()
        result = profiler.runcall(func, *args, **kwargs)
        profiler.dump_stats(profile_filename)
        return result
    return wrapper

strided_slice_40:0 strided_slice_39:0 strided_slice_38:0 strided_slice_33:0
strided_slice_40:0 strided_slice_39:0 strided_slice_38:0 strided_slice_33:0


In [388]:
pool = np.load('../prepare_dataset/pool_00.npz')
features_bool, f_time, label = (pool[name] for name in ['features', 'f_time', 'label'])

In [389]:
np.random.seed(42)
features = np.random.normal(loc=features_bool*1.0, scale=1.0)
#features = features[:299999,:]; label = label[:299999, :]
features_swap = np.concatenate([features[:,7:8], features[:,1:7], features[:,0:1], features[:,8:]], axis=1)

In [390]:
%%time
make_split(np.zeros(label.shape), features, label)

CPU times: user 116 ms, sys: 55.9 ms, total: 172 ms
Wall time: 234 ms


{'avg_current_loss': array([0.69314718]),
 'best_avg_loss': 0.6124402703897527,
 'best_delta_down': -0.8534899433501377,
 'best_delta_up': -0.7958979221951163,
 'best_feature_index': 11,
 'best_index': 261521,
 'best_loss': 183732.0811169258,
 'current_loss': array([207944.15416798]),
 'thr': 1.3039966661872284}

In [391]:
%%time
make_split(np.zeros(label.shape), features_swap, label)

CPU times: user 120 ms, sys: 35.6 ms, total: 156 ms
Wall time: 233 ms


{'avg_current_loss': array([0.69314718]),
 'best_avg_loss': 0.6124402703897527,
 'best_delta_down': -0.8534899433501377,
 'best_delta_up': -0.7958979221951163,
 'best_feature_index': 11,
 'best_index': 261521,
 'best_loss': 183732.0811169258,
 'current_loss': array([207944.15416798]),
 'thr': 1.3039966661872284}

In [392]:
class EMatrix:
    def __init__(self, features, label, bias=None, gax=None, splitgax = False):
        self.bias = bias
        self.features = features
        self.label = label
        self.gax = gax
        self.splitgax = splitgax

In [393]:
class LeafData:
    def __init__(self, info):
        self.val = info['prediction'] * info['learning_rate']
        self.train_size = info['ematrix'].label.shape[0]
        self.avg_target = np.mean(info['ematrix'].label, axis=0)[0]
        
    def to_text(self, floatformat = '.6f'):
        return ('{:'+ floatformat + '} ({})\n({:'+floatformat+'})').format(self.val, self.train_size, self.avg_target)
    
    def shape(self):
        return 'box'

class SplitData:
    def __init__(self, val):
        self.val = val
        
    def to_text(self, floatformat = '.4f'):
        return ('f_{{{ind}}} < {thr:'+floatformat+'}').format(ind=self.val['best_feature_index'], thr=self.val['thr'])
    
        
class TreeNode:
    def __init__(self):
        self.left = None
        self.right = None
        self.depth = 0
        self.val = None
        self.id = None
        
    def to_text(self, floatformat = '.6f'):
        return self.val.to_text(floatformat)
    
    def shape(self):
        return 'circle'
        

In [394]:
def init_id_helper(node, current_id):
    node.id = current_id[0]
    current_id[0] += 1
    if not isinstance(node, TreeNode):
        return
    init_id_helper(node.left, current_id)
    init_id_helper(node.right, current_id)

def init_id(root):
    current_id = [0]
    init_id_helper(root, current_id)
    return current_id[0]

In [395]:
def init_arrays_helper(node, arrays):
    if not isinstance(node, TreeNode):
        arrays['is_leaf'][node.id] = 1
        arrays['leaf_data'][node.id, 0] = node.val  # Leaf
        return
    init_arrays_helper(node.left, arrays)
    init_arrays_helper(node.right, arrays)
    arrays['yes_node'][node.id] = node.left.id
    arrays['no_node'][node.id] = node.right.id
    arrays['thresholds'][node.id] = node.val.val['thr']
    arrays['features'][node.id] = node.val.val['best_feature_index']
    arrays['is_leaf'][node.id] = 0
    arrays['depths'][node.id] = node.depth
    
def init_arrays(root, n):
    def empty_array():
        return np.zeros(n, dtype=np.int32)
    arrays = dict(features=empty_array(),
                  thresholds=np.zeros(n, dtype=np.float32),
                  yes_node=empty_array(),
                  no_node=empty_array(),
                  is_leaf=empty_array(),
                  depths=empty_array(),
                  leaf_data=np.zeros((n,1), dtype=np.float32)
                 )
    init_arrays_helper(root, arrays)
    arrays['treedepth'] = np.max(arrays['depths'])
    return arrays

In [396]:
def prior_finish(params, info, parent):
    if parent is None:
        return False
    return params['max_depth'] <= parent.depth

def post_finish(params, info, split_info, parent):
    if split_info['left_info']['ematrix'].label.shape[0] < 2:
        return True
    if split_info['right_info']['ematrix'].label.shape[0] < 2:
        return True
    return False

In [404]:
def new_ax(ax, cond):
    reindex = np.cumsum(cond)-1
    axT = ax.T
    return reindex[axT][cond[axT]].reshape((ax.shape[1], -1)).T
time1 = 0
time2 = 0
time3 = []
def split_ematrix(ematrix, depth, sess=None):
    global time1, time2, time3
    if ematrix.gax is not None:
        start = time.clock()
        #print(ematrix.features.shape, file=sys.stderr)
        #if (ematrix.features.shape[0]<=2):
        #    print(ematrix.gax, file=sys.stderr)
        split_info = make_split_quick(ematrix.bias, ematrix.features, ematrix.label, ematrix.gax, 
                                      profile_file = ('profile_cumsum2.json' if depth==1 else None), sess=sess)
        dif = time.clock() - start
        time1 += dif
        time3 += [dif]
        
        cond_left = split_info['left_cond']
        cond_right = split_info['right_cond']
        ax_left = split_info['ax_left']
        ax_right = split_info['ax_right']
    else:
        split_info = make_split(ematrix.bias, ematrix.features, ematrix.label)
        cond_left = best_feature < split_info['thr']
        cond_right = np.logical_not(cond_left)
        ax_left = ax_right = None

        
    start = time.clock()
    features = ematrix.features
    bias = ematrix.bias
    label = ematrix.label
        
    if False:
        thr = split_info['thr']        

        best_feature = features[:, split_info['best_feature_index']] 

        cond_left = best_feature < thr
        if not np.all((cond_left == split_info['left_cond']).ravel()):
                print("AAAA", file=sys.stderr)
        cond_right = np.logical_not(cond_left)
        if ematrix.gax is not None:
            ax_left = new_ax(ematrix.gax, cond_left)
            ax_right = new_ax(ematrix.gax, cond_right)
            if not np.all((ax_left == split_info['ax_left']).ravel()):
                print("AAAA", file=sys.stderr)
            if not np.all((ax_right == split_info['ax_right']).ravel()):
                print("BBBB", file=sys.stderr)
        else:
            ax_left = ax_right = None
    
    
    left_ematrix = EMatrix(features[cond_left, :], label[cond_left], bias[cond_left], gax=ax_left)
    right_ematrix = EMatrix(features[cond_right, :], label[cond_right], bias[cond_right], gax=ax_right)
    left_info = {'prediction': split_info['best_delta_up'], 'ematrix': left_ematrix, 'sess': sess}
    right_info = {'prediction': split_info['best_delta_down'], 'ematrix': right_ematrix, 'sess': sess}
    split_info['left_info'] = left_info
    split_info['right_info'] = right_info
    time2 += time.clock() - start
    return left_info, right_info, split_info

In [405]:
def build_tree_helper(params, info, parent):
    info['learning_rate'] = params['learning_rate']
    if parent and parent.depth < 6:
        print("{d}".format(d=parent.depth) 
              if parent else '---',
              "".format(shape=info['ematrix'].label.shape[0]),
              end=' ', file=sys.stderr)
    if prior_finish(params, info, parent):
        return LeafData(info)
    node = TreeNode()
    node.depth = parent.depth + 1 if parent else 1
    
    left_info, right_info, split_info = split_ematrix(info['ematrix'], node.depth, sess=info['sess'])
    if post_finish(params, info, split_info, parent):
        #print(split_info['right_info']['ematrix'].label.shape[0])
        #print(split_info)
        return LeafData(info)

    node.val = SplitData(split_info)
    node.left = build_tree_helper(params, left_info, node)
    node.right = build_tree_helper(params, right_info, node)  
    return node

In [406]:
@profile
def build_tree(params, ematrix, sess = None):
    info = {'ematrix': ematrix, 'sess': sess}
    if ematrix.splitgax and ematrix.gax is None:
        ematrix.gax = make_gax(ematrix.features)
    return build_tree_helper(params, info=info, parent=None)

In [407]:
def tree_apply(tree_arrays, features):
    qi = np.zeros(features.shape[0], dtype=np.int32)
    for current_depth in range(tree_arrays['treedepth']):
        fi = tree_arrays['features'][qi]
        f = np.choose(fi, features.T)
        t = tree_arrays['thresholds'][qi]
        #print(qi, fi, f, t)
        #if current_depth == 0: 
        #    print(fi, f.shape, features.shape, f)
        answer = (f < t)*1
        new_qi = answer*tree_arrays['yes_node'][qi] + (1-answer)*tree_arrays['no_node'][qi]
        qi = new_qi
    leaf_data = tree_arrays['leaf_data'][qi, 0]
    return leaf_data

In [408]:
features.shape, label.shape

((300000, 15), (300000, 1))

In [409]:
global_ax = make_gax(features)

In [418]:
%%time
start_params = {'max_depth': 8, 'learning_rate':0.3}
tf.constant
time1 = time2 =0
time3 = []
with tf.Session(graph=split_quick_graph) as s:
    tree = build_tree(start_params, EMatrix(features, label, np.zeros(label.shape), gax=global_ax, splitgax=True), sess=s)
print("\nsplit time = {:.5f}, {:.5f}, [{}]".format(time1, time2, ", ".join("{:.5f}".format(tt) for tt in time3)), file=sys.stderr)
tree_arrays = init_arrays(tree, init_id(tree))

1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  1  2  3  4  5  5  4  5  5  3  4  4  5  

CPU times: user 7.51 s, sys: 408 ms, total: 7.92 s
Wall time: 2.97 s


5  2  3  3  4  5  5  4  
split time = 7.69484, 0.18254, [0.83943, 0.44301, 0.30604, 0.24914, 0.18018, 0.18160, 0.14574, 0.04677, 0.13815, 0.04391, 0.02334, 0.02754, 0.01795, 0.01573, 0.01552, 0.01553, 0.01810, 0.01652, 0.01855, 0.08165, 0.08186, 0.05527, 0.05266, 0.01934, 0.03673, 0.01615, 0.04129, 0.01751, 0.01696, 0.01588, 0.01704, 0.01778, 0.01408, 0.01408, 0.10786, 0.09374, 0.06516, 0.05852, 0.05292, 0.01846, 0.01825, 0.01759, 0.01455, 0.04810, 0.01715, 0.01905, 0.01496, 0.04832, 0.04680, 0.01647, 0.02319, 0.02024, 0.02044, 0.01620, 0.02218, 0.01715, 0.01680, 0.01460, 0.02146, 0.01502, 0.01663, 0.01519, 0.02089, 0.01667, 0.01692, 0.22593, 0.20824, 0.01871, 0.01519, 0.01407, 0.01425, 0.01663, 0.01625, 0.01781, 0.01833, 0.02263, 0.02017, 0.01740, 0.01821, 0.01539, 0.01639, 0.01638, 0.21032, 0.14443, 0.14955, 0.13381, 0.02898, 0.01774, 0.01575, 0.01480, 0.07996, 0.07562, 0.06527, 0.03184, 0.01719, 0.01965, 0.01712, 0.02437, 0.02131, 0.02041, 0.01798, 0.01546, 0.01618, 0.01527, 0.01551

In [421]:
import pstats
p = pstats.Stats('build_tree.prof')
#p.sort_stats('tottime').print_stats()

In [422]:
pred = tree_apply(tree_arrays, features)
auc1 = metrics.roc_auc_score(label[:, 0], pred)
auc0 = metrics.roc_auc_score(label[:, 0], np.zeros(label.shape[0]))
auc1, auc0

(0.5296545537774653, 0.5)

In [423]:
#tree.left.left.left.left.val, tree.val
tree.left.left

In [424]:
def tree2gv(tree):
    result = graphviz.Graph('ni')
    #result.attr(size='12,0')
    tree2gv_helper(tree, result, '')
    return result

In [425]:
def tree2gv_helper(node, result, id):
    idn = id
    result.node(idn, node.to_text(), shape='box') # node.shape())
    if isinstance(node, LeafData):
        return
    if node.left is not None:
        idl = id + '0'
        tree2gv_helper(node.left, result, idl)
        result.edge(idn, idl)
    if node.right is not None:
        idr = id + '1'
        tree2gv_helper(node.right, result, idr)
        result.edge(idn, idr)
    
    

In [426]:
#tree2gv(tree)

In [427]:
class EBooster:
    def __init__(self, forest):
        self.forest = forest
    
    def predict(self, features, tree_limit = None):
        pred = np.zeros(features.shape[0], dtype=np.float32)
        for tree, tree_arrays in (self.forest if tree_limit is None else self.forest[:tree_limit]):
            pred = pred + tree_apply(tree_arrays, features)
        return pred 
    
def train(params, ematrix, num_boost_round = 10):
    start_params = {'max_depth': 5, 'learning_rate': 0.3, 'splitgax': False} 
    start_params.update(params)
    
    if start_params['splitgax'] and ematrix.gax is None:
        ematrix.gax = make_gax(ematrix.features)
    
    forest = []
    bias = np.zeros(ematrix.label.shape)
    for r in range(num_boost_round):
        print("\n{} round".format(r), file=sys.stderr)
        tree = build_tree(start_params, EMatrix(ematrix.features, ematrix.label, bias, gax=ematrix.gax, splitgax=start_params['splitgax']))
        #print("tree ok, bias shape = {}".format(bias.shape), file=sys.stderr)
        tree_arrays = init_arrays(tree, init_id(tree))
        bias_delta = tree_apply(tree_arrays, features)
        #print("apply ok, bias delta shape = {}".format(bias_delta.shape), file=sys.stderr)
        bias = bias + np.reshape(bias_delta, newshape=bias.shape)
        forest.append((tree, tree_arrays))
        #print("forest appended", file=sys.stderr)
        
    return EBooster(forest)

In [428]:
%%time
boost_params = {'max_depth': 8, 'learning_rate':0.3, 'splitgax': True}
booster = train(boost_params, EMatrix(features, label), num_boost_round=12)


0 round
1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  1  2  3  4  5  5  4  5  5  3  4  4  5  5  2  3  3  4  5  5  4  
1 round
1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  3  4  5  5  4  5  5  
2 round
1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  
3 round
1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  1  2  3  4  5  5  4  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  
4 round
1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  1  2  3  4  5  5  4  5  5  3  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  
5 round
1  2  3  4  5  5  4  5  5  3  4  5  5  4  5  5  2  3  4  5  5  4  5  5  3  4

CPU times: user 7min 58s, sys: 10.9 s, total: 8min 9s
Wall time: 7min 7s


In [26]:
#%%time
#gax13 = make_gax(features)
#print(gax13.shape)

In [27]:
pred2 = booster.predict(features)
metrics.roc_auc_score(label[:, 0], pred2)

0.5297163871088377

In [ ]:
pred2b = booster.predict(features, tree_limit=16)
metrics.roc_auc_score(label[:, 0], pred2)

In [ ]:
np.min(pred2b), np.max(pred2b)

In [ ]:
tree2gv(booster.forest[8][0])

In [ ]:
dmat = xgb.DMatrix(features, label)

In [ ]:
%%time
params = {'max_depth': 4, 'objective': 'binary:logistic', 'eta': 0.3, 'tree_method': 'exact', 'lambda': 1}
model = xgb.train(params, dmat, num_boost_round=20)

In [ ]:
pred_fast = model.predict(dmat)
metrics.roc_auc_score(label[:, 0], pred_fast)

In [ ]:
pred_fast = model.predict(dmat, ntree_limit=16)
metrics.roc_auc_score(label[:, 0], pred_fast)

In [ ]:
qq = -np.log(1/pred_fast-1) 
np.min(qq), np.max(qq)

In [ ]:
diff = np.abs(qq - pred2b)
np.mean(diff > 0.1)

In [ ]:
model.dump_model('xgb.dump')

In [ ]:
tree2gv(booster.forest[15][0])

In [ ]:
!cat xgb.dump | tail -n+$(echo 32*11+22 | bc)